In [1]:
!pip install pymongo pyodbc

   ---------------------------------------- 0.0/898.7 kB ? eta -:--:--
   ---------------------------------------- 898.7/898.7 kB 5.8 MB/s eta 0:00:00


In [2]:
import pyodbc
from pymongo import MongoClient

# 1. CONFIGURE CONNECTION STRINGS
SQL_SERVER_CONN_STR = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=NAME\\SQLEXPRESS;"
    "DATABASE=moviepal_db;"
    "Trusted_Connection=yes;"
)
MONGO_CONN_STR = "mongodb://localhost:27017"  # or your Atlas URI

# 2. CONNECT TO SQL SERVER
sql_conn = pyodbc.connect(SQL_SERVER_CONN_STR)
sql_cursor = sql_conn.cursor()

# 3. CONNECT TO MONGODB
mongo_client = MongoClient(MONGO_CONN_STR)
mongo_db = mongo_client["moviepal_db"]  # target MongoDB database name

# 4. GET LIST OF USER TABLES IN THE MOVIEPAL_DB
#    We assume all tables live in 'dbo' schema. Adjust if yours uses a different schema.
sql_cursor.execute("""
    SELECT TABLE_SCHEMA, TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_TYPE = 'BASE TABLE'
      AND TABLE_CATALOG = 'moviepal_db'
      AND TABLE_SCHEMA = 'dbo'
""")
tables = sql_cursor.fetchall()  # list of (schema, table_name)

print("Found tables:", [t.TABLE_NAME for t in tables])

# 5. FOR EACH TABLE, SELECT * AND INSERT INTO MONGODB
for schema, table_name in tables:
    collection = mongo_db[table_name]   # collection name = table name
    print(f"Migrating table {schema}.{table_name} → MongoDB collection '{table_name}'")

    # Build and execute a SELECT * query
    select_sql = f"SELECT * FROM [{schema}].[{table_name}];"
    sql_cursor.execute(select_sql)
    columns = [column[0] for column in sql_cursor.description]

    batch = []
    BATCH_SIZE = 500  # adjust as needed
    rowcount = 0

    for row in sql_cursor:
        # row is a tuple matching the columns list
        doc = {}
        for idx, col_name in enumerate(columns):
            value = row[idx]
            # Optionally handle date/datetime conversion, Decimal, UUID, etc.
            # For example, if value is a Decimal, you might do: float(value)
            doc[col_name] = value

        batch.append(doc)
        rowcount += 1

        # Bulk‐insert every BATCH_SIZE documents
        if len(batch) >= BATCH_SIZE:
            collection.insert_many(batch)
            batch.clear()

    # Insert any remainder
    if batch:
        collection.insert_many(batch)

    print(f"  → Inserted {rowcount} documents into '{table_name}'.")

# 6. CLEANUP
sql_cursor.close()
sql_conn.close()
mongo_client.close()

print("Migration complete!")


OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: Error Locating Server/Instance Specified [xFFFFFFFF].  (-1) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (-1)')